In [21]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.0'
spark_version = 'spark-3.0.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://security.ubuntu.com/ubuntu bionic-security InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:9 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Reading package lists... Done


In [22]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2020-11-08 22:02:37--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.1’

postgresql-42.2.16. 100%[===================>] 979.38K  5.42MB/s    in 0.2s    

2020-11-08 22:02:38 (5.42 MB/s) - ‘postgresql-42.2.16.jar.1’ saved [1002883/1002883]



In [23]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [24]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Automotive_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   36075342| RAB23OVFNCXZQ|B00LPRXQ4Y|     339193102|17" 2003-2006 For...|      Automotive|          1|            0|          0|   N|                Y|     As it was used,|As it was used, t...| 2015-08-31|
|         US|   42462164|R3NORADVJO6IE6|B000C7S0TO|     907684644|Spectra Premium C...|      Automotive|          5|    

In [40]:
# Create the vine_table DataFrame
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RAB23OVFNCXZQ|          1|            0|          0|   N|                Y|
|R3NORADVJO6IE6|          5|            0|          0|   N|                Y|
|R299F4SO98S5OO|          5|            1|          1|   N|                Y|
|R2DA9DOT03UW6I|          5|            2|          3|   N|                Y|
|R2OGCH681EQHU6|          5|            0|          0|   N|                Y|
| R2JMKIC16MHD7|          5|            2|          2|   N|                Y|
|R1DB5DA7CWWTI8|          5|            0|          0|   N|                Y|
|R1N8XWFDK4QACP|          5|            0|          0|   N|                Y|
|R19VFLX6MRU0CX|          5|            0|          0|   N|                Y|
| RLTD8YDGTP9PT|          1|            0|          0|   N|     

In [41]:
vine_df_20 = vine_df.filter(vine_df["total_votes"] >= "20")
vine_df_20.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1T4FVSABO0IDP|          5|            8|         31|   N|                Y|
|R397VXR1GAK6C9|          5|           21|         21|   N|                Y|
|R2H6KB9RHS17GA|          3|           59|         69|   N|                Y|
|R3HAK0MUN9F5IT|          1|            6|         28|   N|                Y|
|R29UTZTOPUVRSV|          5|           21|         21|   N|                Y|
| RY9FLUD8VG6XD|          4|           27|         32|   N|                Y|
|R3Q5NAY8BGAKHI|          5|           34|         40|   N|                Y|
| RRVJ7TYDEK7EO|          5|           30|         30|   N|                Y|
|R2S4CHISQA2WDH|          5|           17|         20|   N|                Y|
|R1JNALJ77OULPH|          4|           57|         65|   N|     

In [42]:
new_vine_df = vine_df_20.withColumn("vote_calc", vine_df_20["helpful_votes"]/ vine_df_20["total_votes"]).filter("vote_calc>=0.5")
new_vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|         vote_calc|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R397VXR1GAK6C9|          5|           21|         21|   N|                Y|               1.0|
|R2H6KB9RHS17GA|          3|           59|         69|   N|                Y| 0.855072463768116|
|R29UTZTOPUVRSV|          5|           21|         21|   N|                Y|               1.0|
| RY9FLUD8VG6XD|          4|           27|         32|   N|                Y|           0.84375|
|R3Q5NAY8BGAKHI|          5|           34|         40|   N|                Y|              0.85|
| RRVJ7TYDEK7EO|          5|           30|         30|   N|                Y|               1.0|
|R2S4CHISQA2WDH|          5|           17|         20|   N|                Y|              0.85|
|R1JNALJ77OULPH|          4|  

In [51]:
total_vine_reviews = new_vine_df.filter(new_vine_df["vine"] == "Y").count()
total_vine_reviews

82

In [52]:
total_nonvine_reviews = new_vine_df.filter(new_vine_df["vine"] == "N").count()
total_nonvine_reviews

24742

In [53]:
total_reviews = new_vine_df.count()
total_reviews

24824

In [54]:
total_five_star_reviews = new_vine_df.filter(new_vine_df["star_rating"] == 5).count()
total_five_star_reviews

12840

In [55]:
total_vine_five_star = new_vine_df.filter((new_vine_df["vine"] == "Y")&(new_vine_df["star_rating"] == 5)).count()
total_vine_five_star

33

In [56]:
total_nonvine_five_star = new_vine_df.filter((new_vine_df["vine"] == "N")&(new_vine_df["star_rating"] == 5)).count()
total_nonvine_five_star

12807

In [57]:
percent_five_star_vine = total_vine_five_star/total_vine_reviews * 100
percent_five_star_vine

40.243902439024396

In [58]:
percent_five_star_nonvine = total_nonvine_five_star/total_nonvine_reviews * 100
percent_five_star_nonvine

51.76218575701237